In [ ]:
!pip install nltk

In [ ]:
import nltk
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> all


       | 
       | Downloading package abc to /root/nltk_data...
       |   Unzipping corpora/abc.zip.
       | Downloading package alpino to /root/nltk_data...
       |   Unzipping corpora/alpino.zip.
       | Downloading package averaged_perceptron_tagger to
       |     /root/nltk_data...
       |   Unzipping taggers/averaged_perceptron_tagger.zip.
       | Downloading package averaged_perceptron_tagger_ru to
       |     /root/nltk_data...
       |   Unzipping taggers/averaged_perceptron_tagger_ru.zip.
       | Downloading package basque_grammars to /root/nltk_data...
       |   Unzipping grammars/basque_grammars.zip.
       | Downloading package bcp47 to /root/nltk_data...
       | Downloading package biocreative_ppi to /root/nltk_data...
       |   Unzipping corpora/biocreative_ppi.zip.
       | Downloading package bllip_wsj_no_aux to /root/nltk_data...
       |   Unzipping models/bllip_wsj_no_aux.zip.
       | Downloading package book_grammars to /root/nltk_data...
       |   Un


---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


True

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df= pd.read_csv("/content/abcnews-date-text.csv")

In [ ]:
df.head()

,publish_date,headline_text
0,20030219,aba decides against community broadcasting lic...
1,20030219,act fire witnesses must be aware of defamation
2,20030219,a g calls for infrastructure protection summit
3,20030219,air nz staff in aust strike for pay rise
4,20030219,air nz strike to affect australian travellers


In [ ]:
df.shape

(468991, 3)

In [ ]:
df=df.iloc[:10000,:] #Slice the dataframe

In [ ]:
df.columns

Index(['publish_date', 'headline_text', 'text_clean'], dtype='object')

In [ ]:
from nltk.corpus import stopwords
from string import punctuation
from nltk.probability import FreqDist
from nltk.stem.wordnet import WordNetLemmatizer
from gensim import corpora
import gensim

In [ ]:
lemma = WordNetLemmatizer()

In [ ]:
StopWords= stopwords.words("english")

In [ ]:
#Text Preprocessing
def clean(raw, stopwords,punctuation):
    text_tokenize= nltk.tokenize.word_tokenize(raw)
    punctuation=list(punctuation)
    #Remove punctuations
    text_punc=[]
    for word in text_tokenize:
        if word not in punctuation:
            text_punc.append(word)

    #Remove stop words

    text_stop=[]
    for word in text_punc:
        if word not in stopwords:
            text_stop.append(word.lower())
    text_clean=" ".join(text_stop)
    return text_clean

In [ ]:
df["text_clean"]= df["headline_text"].apply(lambda x: [i.lower() for i in x.split() if i not in StopWords])
df["text_clean"]= df["text_clean"].apply(lambda x: [i for i in x if i not in punctuation])
df["text_clean"]= df["text_clean"].apply(lambda x: [lemma.lemmatize(i) for i in x ])

<ipython-input-36-23aeecb6e9c5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["text_clean"]= df["headline_text"].apply(lambda x: [i.lower() for i in x.split() if i not in StopWords])
<ipython-input-36-23aeecb6e9c5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["text_clean"]= df["text_clean"].apply(lambda x: [i for i in x if i not in punctuation])
<ipython-input-36-23aeecb6e9c5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index

In [ ]:
df.head(10)

,publish_date,headline_text,text_clean
0,20030219,aba decides against community broadcasting lic...,"[aba, decides, community, broadcasting, licence]"
1,20030219,act fire witnesses must be aware of defamation,"[act, fire, witness, must, aware, defamation]"
2,20030219,a g calls for infrastructure protection summit,"[g, call, infrastructure, protection, summit]"
3,20030219,air nz staff in aust strike for pay rise,"[air, nz, staff, aust, strike, pay, rise]"
4,20030219,air nz strike to affect australian travellers,"[air, nz, strike, affect, australian, traveller]"
5,20030219,ambitious olsson wins triple jump,"[ambitious, olsson, win, triple, jump]"
6,20030219,antic delighted with record breaking barca,"[antic, delighted, record, breaking, barca]"
7,20030219,aussie qualifier stosur wastes four memphis match,"[aussie, qualifier, stosur, waste, four, memph..."
8,20030219,aust addresses un security council over iraq,"[aust, address, un, security, council, iraq]"
9,20030219,australia is locked into war timetable opp,"[australia, locked, war, timetable, opp]"


In [ ]:
dictionary = corpora.Dictionary(df["text_clean"])

In [ ]:
print(dictionary)

Dictionary<8656 unique tokens: ['aba', 'broadcasting', 'community', 'decides', 'licence']...>


In [ ]:
doc_term_matrix = [dictionary.doc2bow(row) for row in df["text_clean"]]

In [ ]:
#doc_term_matrix

In [ ]:
Lda = gensim.models.ldamodel.LdaModel
ldamodel = Lda(doc_term_matrix, num_topics = 5, id2word = dictionary, passes=40)